# Horcrux Joystick 입력 학습 진행

## 필요 패키지 import

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd

# 조이스틱 환경 삽입
import horcrux_terrain_v2
from horcrux_terrain_v2.envs import PlaneJoyWorld

# Ray 패키지 삽입
import ray
import os
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.sac import SACConfig

from ray.tune.registry import register_env

import mediapy as media

from scipy.ndimage import uniform_filter1d
from scipy.spatial.transform import Rotation

import matplotlib.pyplot as plt

from gymnasium.utils.save_video import save_video

from IPython.display import Video

# 필요 함수 정의

In [2]:
def get_unique_filename(base_path, ext=".mp4"):
    """중복된 파일명이 존재하면 숫자를 증가하여 새로운 경로를 반환"""
    if not base_path.endswith(ext):
        base_path += ext  # 확장자 자동 추가

    file_name, file_ext = os.path.splitext(base_path)  # 파일명과 확장자 분리
    count = 0
    new_path = f"{file_name}-episode-0"+file_ext

    while os.path.exists(new_path):  # 파일 존재 여부 확인
        new_path = f"{file_name}{count}-episode-0{file_ext}"
        count += 1


    return f"rl-video{count-1}", new_path


def default_plot(x, y, f_name='default_plot', legends=['acc_x', 'acc_y', 'acc_z'], title=''):
    colors = plt.get_cmap("tab10").colors
    fig, ax = plt.subplots(figsize=(15/2.54, 10/2.54))
    ax.set_facecolor((0.95, 0.95, 0.95)) 

    n_column = len(np.shape(y))
    if n_column>2:
        print("The dimmension of data must be less than 3. (1D or 2D)")
        return -1
    
    n_data = np.shape(y)[1]

    for i in range(n_data):
        # **Plot**
        ax.plot(x, y[:,i], linewidth=1.5, linestyle="-", color=colors[i], label=legends[i])
        # ax.plot(x, y[:,i], linewidth=1.5, linestyle="-", color=colors[1], label=legends[1])
        # ax.plot(x, y[:,i], linewidth=1.5, linestyle="-", color=colors[2], label=legends[2])

    # **Grid 설정**
    ax.grid(True, linestyle="--", linewidth=1, color="#202020", alpha=0.7)  # 주요 그리드
    ax.minorticks_on()
    ax.grid(True, which="minor", linestyle=":", linewidth=0.5, color="#404040", alpha=0.5)  # 보조 그리드

    # **Axis 스타일 설정**
    ax.spines["top"].set_linewidth(1.0)
    ax.spines["right"].set_linewidth(1.0)
    ax.spines["left"].set_linewidth(1.0)
    ax.spines["bottom"].set_linewidth(1.0)

    ax.tick_params(axis="both", labelsize=11, width=1.0)  # 폰트 크기 및 라인 두께
    ax.xaxis.label.set_size(12)
    ax.yaxis.label.set_size(12)

    # **폰트 및 제목 설정**
    plt.rcParams["font.family"] = "Arial"
    ax.set_xlabel("X-Axis", fontsize=12, fontweight="bold")
    ax.set_ylabel("Y-Axis", fontsize=12, fontweight="bold")
    ax.set_title(title, fontsize=14, fontweight="bold")

    # **Legend (MATLAB 스타일 적용)**
    ax.legend(loc="upper right", ncol=3, fontsize=10, frameon=True)

    # **비율 설정 (MATLAB의 `pbaspect([2.1 1 1])`과 비슷한 효과)**
    fig.set_size_inches(2.1 * 5, 5)  # 비율 2.1:1 (기본 높이 5inch 기준)

    # **Save Figure (MATLAB saveas와 유사)**
    plt.savefig(f"./figs/{f_name}.png", dpi=600, bbox_inches="tight")

    plt.show()

def moving_average(data, window_size):
    kernel = np.ones(window_size) / window_size
    return np.convolve(data, kernel, mode='same')  # 'valid'는 경계 제외

## Ray 실행

In [3]:
ray.init(dashboard_host="0.0.0.0", dashboard_port=8265)

2025-03-25 19:37:15,337	INFO worker.py:1810 -- Started a local Ray instance. View the dashboard at http://10.130.6.78:8265 


Python version:,3.12.9
Ray version:,2.39.0
Dashboard:,http://10.130.6.78:8265


## Gym 환경 등록하기

In [4]:
env_config = {
    "forward_reward_weight": 60.0,
    "rotation_reward_weight": 40.0,
    "unhealthy_max_steps": 100.0,
    "healthy_reward": 3.0,
    "healthy_roll_range": (-35,35),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 1.5,
    "termination_reward": 0,
    "gait_params": (30, 30, 40, 40, 0),
    "use_friction_chg": True,
    "joy_input_random": True,
    "use_imu_window": True,
    "ctrl_cost_weight": 1.5,
}

render_env_config = {
    "forward_reward_weight": 60.0,
    "rotation_reward_weight": 40.0,
    "unhealthy_max_steps": 100.0,
    "healthy_reward": 3.0,
    "healthy_roll_range": (-35,35),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 1.5,
    "termination_reward": 0,
    "gait_params": (30, 30, 40, 40, 0),
    "use_friction_chg": True,
    "joy_input_random": True,
    "render_mode": "rgb_array",
    "render_camera_name": 'ceiling',
    "use_imu_window": True,
    "ctrl_cost_weight": 1.5,
}

# env = gym.make("horcrux_terrain_v2/plane-v2", **render_env_config)

# JoyWorld
register_env("joy-v1", lambda config: PlaneJoyWorld( forward_reward_weight=env_config["forward_reward_weight"], 
                                                     rotation_reward_weight=env_config["rotation_reward_weight"], 
                                                     unhealthy_max_steps=env_config["unhealthy_max_steps"],
                                                     healthy_reward=env_config["healthy_reward"], 
                                                     healthy_roll_range=env_config["healthy_roll_range"],
                                                     terminating_roll_range=env_config["terminating_roll_range"],
                                                     rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                     termination_reward=env_config["termination_reward"],
                                                     gait_params=env_config["gait_params"],
                                                     use_friction_chg=env_config["use_friction_chg"],
                                                     joy_input_random=env_config["joy_input_random"],
                                                     use_imu_window=env_config["use_imu_window"],
                                                     ctrl_cost_weight=env_config["ctrl_cost_weight"],
                                                   )
            )

## 학습 알고리즘 설정하기

In [5]:
config = SACConfig()

# 구형 API 구조 사용
config.api_stack(
    enable_rl_module_and_learner=False,
    enable_env_runner_and_connector_v2=False,
)

config.environment("joy-v1")
config.framework("torch")

# 병렬 CPU 사용 설정
total_workers = 16
config.resources(num_gpus=1)
config.env_runners(num_env_runners = total_workers, num_cpus_per_env_runner = 1, num_gpus_per_env_runner = 0.3/(total_workers), rollout_fragment_length = 1000)
config.training(
    gamma=0.95,
    replay_buffer_config={
    "_enable_replay_buffer_api": True,
    "capacity": int(1000000),
    "type": "MultiAgentReplayBuffer",
    "replay_batch_size": 10000,
    # If True prioritized replay buffer will be used.
    # "prioritized_replay": False,
    # "prioritized_replay_alpha": 0.6,
    # "prioritized_replay_beta": 0.4,
    # "prioritized_replay_eps": 1e-6,
    # Whether to compute priorities already on the remote worker side.
    # "worker_side_prioritization": False,
    },

    q_model_config = {
            "fcnet_hiddens": [100, 100, 256, 256, 100, 32],
            "fcnet_activation": "tanh",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "custom_model": None,  # Use this to define custom Q-model(s).
            "custom_model_config": {},
    },
    policy_model_config = {
            "fcnet_hiddens": [100, 100, 256, 256, 100, 32],
            "fcnet_activation": "tanh",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "custom_model": None,  # Use this to define a custom policy model.
            "custom_model_config": {},
    },

    train_batch_size = 100000,
    num_steps_sampled_before_learning_starts = 200000,
)

algo = config.build()


/home/bong/anaconda3/envs/gd239/lib/python3.12/site-packages/ray/rllib/algorithms/algorithm.py:567: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/bong/anaconda3/envs/gd239/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/bong/anaconda3/envs/gd239/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is depreca

## 학습시작

In [ ]:
from pprint import pprint
import datetime

n_iter = 8500
save_iter = 0
save_name = "SAC_layer_small_32_friction_joy_imu_325_record"

for i in range(n_iter):
    result = algo.train()
    print(f"{i:03d}", end=", ")
    # result.pop("config")
    # pprint(result)

    if i%100 == 0:
        checkpoint_dir = algo.save(save_name+"_"+str(save_iter))
        print(f"Checkpoint saved in directory {checkpoint_dir}")
        save_iter += 1


        # Record Validation Env
        env = gym.make("horcrux_terrain_v2/plane-v2", **render_env_config)
        obs = env.reset()[0]
        env_done = False

        _video_base_name = 'rl-video'

        frames = []
        info = []

        for i in range(1000):
            act = algo.compute_single_action(observation=obs)
            obs, _, _, env_done, env_info = env.step(act)
            pixels = env.render()
            frames.append(pixels)
            info.append(env_info)

        _f_name, _full_path = get_unique_filename(f"./video/{_video_base_name}")
        save_video(frames, "./video/", name_prefix=_f_name, fps=env.metadata['render_fps'])

        _f_video_info = open(f"./video/joy_input.txt", 'a')
        _f_video_info.write(f'File creation time: {datetime.datetime.now()}\n')
        _f_video_info.write(f'Video file name: {_f_name}, Joy input: {info[0]["joy_input"]}, Friction: {info[0]["friction_coeff"]}\n')
        _f_video_info.close()
        env.close()


algo.save(save_name+str("_final"))

000, Checkpoint saved in directory TrainingResult(checkpoint=Checkpoint(filesystem=local, path=SAC_layer_small_32_friction_joy_imu_325_record_0), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {}, 'num_env_steps_sampled': 16000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 16000, 'num_agent_steps_trained': 0}, 'env_runners': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_timesteps_total': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}, 'num_episodes': 0, 'episode_return_max': nan, 'episode_return_min': nan, 'episode_return_mean': nan, 'episodes_this_iter': 0}, 'num_healthy_workers': 16, 'num_in_flight_async_sample_reqs': 0, 'num_remote_worker_restarts': 0, 'num_agent_steps_sample

2025-03-25 19:40:55,170	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


012, 013, 014, 015, 016, 017, 018, 019, 020, 021, 022, 023, 

# 학습 알고리즘 평가하기

In [ ]:
np.round(info[0]['joy_input'],2)

In [ ]:
algo = Algorithm.from_checkpoint("./SAC_layer_512_5_32_linear_friction_joy2_2") # 학습된 정책 로드

## 평가용 Env 생성 및 실행

In [ ]:
import pathlib
import torch
import numpy as np
import gymnasium as gym
from horcrux_terrain_v1.envs import SandWorld
from ray.rllib.algorithms.algorithm import Algorithm
import time

eval_config = env_config
eval_config["use_friction_chg"] = False
eval_config["joy_input_random"] = False
eval_config["joy_input"] = (1, 0, 0)


In [ ]:
env = gym.make("horcrux_terrain_v1/plane-v2", 
               terminate_when_unhealthy = False, 
               render_mode = "human", 
               render_camera_name = 'ceiling', 
               use_gait = True,               
               **eval_config,
               ) 

for j in range(3):
   episode_return = 0
   yaw_total = 0
   terminated = truncated = False

   obs, info = env.reset()

   for i in range(1000):

      action = algo.compute_single_action(observation= obs)
      
      obs, reward, terminated, truncated, info = env.step(action)
      if i == 0:
          print(f"Joy input = {info['joy_input']}")
      if terminated:
         print("terminated")

      episode_return += reward
      yaw_total += info['step_ypr'][0] / 0.1

   print(f"Reached episode return of {episode_return}.")
   print(yaw_total)

env.close()

### Pytorch 모델 바로 불러오기

In [ ]:
import torch
from ray.rllib.policy.policy import Policy
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2

_algo = Algorithm.from_checkpoint("./Paper_agents/good/Linear/SAC_layer_512_5_32_Linear_restart_final")
# _policy = Policy.from_checkpoint("./SAC_layer_512_5_32_linear_friction_joy_31_0")
# _policy = Policy.from_checkpoint("./Paper_agents/good/Linear/SAC_layer_512_5_32_Linear_restart_final")


In [ ]:
randobs = np.random.random(94)

# _algo.compute_single_action(randobs)
# _algo.get_policy('default_policy').dist_class.mro()
_algo.get_policy('default_policy').model.action_dist



In [ ]:
randobs = np.random.random(94)

# obs = torch.tensor(randobs, dtype=torch.float32)
_policy['default_policy'].dist_class
# _policy['default_policy'].compute_single_action(torch.tensor(randobs, dtype=torch.float32), explore=False)